In [25]:
import os
import pickle
import numpy as np
from collections import Counter
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

In [26]:
# LOAD DATASET
def load_dataset(path="../Dataset/windows_file.pkl"):
    with open(path, "rb") as f:
        data = pickle.load(f)
    print("Loaded samples:", len(data))
    return data


In [27]:
# RESAMPLE SpO2 to match 960 length because it was 120 
def resample_spo2(spo2_signal, target_len=960):
    # Linear interpolation to match respiration length
    x_old = np.linspace(0, 1, len(spo2_signal))
    x_new = np.linspace(0, 1, target_len)
    return np.interp(x_new, x_old, spo2_signal)

In [28]:
# PREPARE TENSORS
def prepare_arrays(dataset):
    X = []
    y = []
    participants = []

    for sample in dataset:
        airflow = sample["airflow"]
        thoracic = sample["thoracic"]
        spo2 = resample_spo2(sample["spo2"], target_len=len(airflow))

        stacked = np.stack([airflow, thoracic, spo2], axis=1)

        label = 0 if sample["label"].lower() == "normal" else 1

        X.append(stacked.astype(np.float32))
        y.append(label)
        participants.append(sample["participant"])

    X = np.array(X)
    y = np.array(y)
    participants = np.array(participants)

    print("X shape:", X.shape)
    print("y distribution:", Counter(y))

    return X, y, participants


In [29]:
# BUILD 1D CNN
def build_cnn(input_shape):
    model = models.Sequential([
        layers.Input(shape=input_shape),   
        # layers.Conv1D(32, 5, activation='relu', input_shape=input_shape),
        layers.Conv1D(32, 5, activation='relu'),
        layers.MaxPooling1D(2),

        layers.Conv1D(64, 5, activation='relu'),
        layers.MaxPooling1D(2),

        layers.Conv1D(128, 3, activation='relu'),
        layers.GlobalAveragePooling1D(),

        layers.Dense(64, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])

    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    return model

In [30]:
# LEAVE-ONE-PARTICIPANT-OUT
def run_lopo_cv(X, y, participants, epochs=5, batch_size=64):
    unique_pids = sorted(set(participants))

    accs, precs, recs = [], [], []

    for test_pid in unique_pids:
        print(f"\nLOPO Fold: Test = {test_pid}")

        train_idx = participants != test_pid
        test_idx = participants == test_pid

        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        model = build_cnn(input_shape=X.shape[1:])

        model.fit(
            X_train, y_train,
            epochs=epochs,
            batch_size=batch_size,
            verbose=0,
            class_weight={0:1, 1:5}
        )

        # save model
        model_path = f"../Models/cnn_test_{test_pid}.keras"
        model.save(model_path)
        print("Saved model to ", model_path)

        y_pred_prob = model.predict(X_test, verbose=0)
        y_pred = (y_pred_prob > 0.5).astype(int).flatten()

        acc = accuracy_score(y_test, y_pred)
        prec = precision_score(y_test, y_pred, zero_division=0)
        rec = recall_score(y_test, y_pred, zero_division=0)
        cm = confusion_matrix(y_test, y_pred)

        print("Accuracy :", acc)
        print("Precision:", prec)
        print("Recall   :", rec)
        print("Confusion Matrix:\n", cm)

        accs.append(acc)
        precs.append(prec)
        recs.append(rec)

    print("FINAL RESULTS (Mean over LOPO)")
    print("Accuracy :", np.mean(accs))
    print("Precision:", np.mean(precs))
    print("Recall   :", np.mean(recs))


In [31]:
# FINAL TRAINING ON FULL DATA AND 
def train_final_model(X, y, epochs=5, batch_size=64):
    print("\n Training FINAL model on full dataset...")

    model = build_cnn(input_shape=X.shape[1:])

    model.fit(
        X, y,
        epochs=epochs,
        batch_size=batch_size,
        verbose=1,
        class_weight={0:1, 1:5}
    )

    os.makedirs("../Models", exist_ok=True)
    final_path = "../Models/final_cnn.keras"
    model.save(final_path)

    print(" Final model saved to", final_path)

In [32]:
if __name__ == "__main__":
    dataset = load_dataset()
    X, y, participants = prepare_arrays(dataset)

    # evaluation
    run_lopo_cv(X, y, participants, epochs=5)

    # final training
    train_final_model(X, y, epochs=5)

Loaded samples: 8800
X shape: (8800, 960, 3)
y distribution: Counter({0: 8038, 1: 762})

LOPO Fold: Test = AP01
Saved model to  ../Models/cnn_test_AP01.keras
Accuracy : 0.8578485181119648
Precision: 0.13716814159292035
Recall   : 0.3263157894736842
Confusion Matrix:
 [[1532  195]
 [  64   31]]

LOPO Fold: Test = AP02
Saved model to  ../Models/cnn_test_AP02.keras
Accuracy : 0.9005087620124365
Precision: 0.2830188679245283
Recall   : 0.09803921568627451
Confusion Matrix:
 [[1578   38]
 [ 138   15]]

LOPO Fold: Test = AP03
Saved model to  ../Models/cnn_test_AP03.keras
Accuracy : 0.08549528301886793
Precision: 0.010217113665389528
Recall   : 0.9411764705882353
Confusion Matrix:
 [[ 129 1550]
 [   1   16]]

LOPO Fold: Test = AP04
Saved model to  ../Models/cnn_test_AP04.keras
Accuracy : 0.9114906832298136
Precision: 0.0
Recall   : 0.0
Confusion Matrix:
 [[1761    2]
 [ 169    0]]

LOPO Fold: Test = AP05
Saved model to  ../Models/cnn_test_AP05.keras
Accuracy : 0.7634408602150538
Precision: 0.